In [1]:
import sklearn.neighbors._base
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
# !pip install impyute
# !pip install fancyimpute
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as skLDA
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from scipy import stats
import numpy as np
# import impyute as impy
from fancyimpute import SoftImpute#, MatrixFactorization
import pandas as pd
import time
import math
from missingpy import MissForest
from sklearn.model_selection import train_test_split
from numpy.linalg import norm, inv

from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from fancyimpute import SimpleFill, SoftImpute
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from imblearn.under_sampling import InstanceHardnessThreshold

from imblearn.metrics import geometric_mean_score
from imblearn.metrics import sensitivity_score

from sklearn.metrics import f1_score, roc_auc_score
from sklearn import metrics
from imblearn.datasets import fetch_datasets
from imblearn.over_sampling import KMeansSMOTE
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SVMSMOTE
from functions.OsImp import *
import warnings
warnings.filterwarnings("ignore")

from functions.eval import *
from functions.utils import *
from functions.OsImp import *
from functions.OsImp_star import *
from functions.dpers import *
from functions.dimv import dimv
# imputer = dimv

imputer = IterativeImputer().fit_transform
# imputer = SoftImpute(convergence_threshold=0.1, max_iters=10).fit_transform
# imputer = MissForest().fit_transform

In [2]:
  df = fetch_datasets()['mammography']
  X, y = df.data, df.target
  le = LabelEncoder()
  y = le.fit_transform(y)
  G = len(np.unique(y))
  print(np.shape(X), G, sum(y==0)*.6*.5, sum(y==1)*.6*.5)
  X[:3]

(11183, 6) 2 3276.9 78.0


array([[ 0.23001961,  5.0725783 , -0.27606055,  0.83244412, -0.37786573,
         0.4803223 ],
       [ 0.15549112, -0.16939038,  0.67065219, -0.85955255, -0.37786573,
        -0.94572324],
       [-0.78441482, -0.44365372,  5.6747053 , -0.85955255, -0.37786573,
        -0.94572324]])

In [3]:
missing_rates = np.array([0.0, 0.1, 0.2, 0.3, 0.4])
res_all = []
for missing_rate in missing_rates:
  K, nMC, result = 2, 10, []
  n = np.array([-200,100])
  res = np.asarray([get_all_res_star(X, y, i, n, K, G,imputer, missing_rate) for i in range(nMC)], dtype = object)
  res = show_result(res)
  res_all.append(res)    

In [4]:
concat_tab = pd.concat((res_all[0][1][0], res_all[0][1][1], 
                        res_all[1][1][0], res_all[1][1][1], 
                        res_all[2][1][0], res_all[2][1][1], 
                        res_all[3][1][0], res_all[3][1][1], 
                        res_all[4][1][0], res_all[4][1][1]))

In [5]:
leftindex = pd.MultiIndex.from_product([["0%", "10%", "20%", "30%", "40%"],
                                        ["F-1 score","sensitivity"], 
                                        ["SVM", "LR", "DCT"]], 
                                       names = ["missing rate", "metric", "classifier"])
all_res = pd.DataFrame(concat_tab.to_numpy(), index = leftindex, columns = concat_tab.columns)
all_res

NearMiss         kmeanSmt  \
missing rate metric      classifier                                     
0%           F-1 score   SVM         0.565$\pm$0.045  0.983$\pm$0.002   
                         LR          0.607$\pm$0.036  0.982$\pm$0.003   
                         DCT         0.447$\pm$0.093  0.981$\pm$0.002   
             sensitivity SVM          0.42$\pm$0.047  0.985$\pm$0.002   
                         LR           0.462$\pm$0.04  0.984$\pm$0.002   
                         DCT         0.315$\pm$0.071  0.981$\pm$0.002   
10%          F-1 score   SVM         0.515$\pm$0.043  0.982$\pm$0.002   
                         LR          0.568$\pm$0.024  0.981$\pm$0.002   
                         DCT           0.426$\pm$0.1  0.979$\pm$0.002   
             sensitivity SVM         0.373$\pm$0.042  0.984$\pm$0.002   
                         LR          0.422$\pm$0.025  0.983$\pm$0.002   
                         DCT            0.3$\pm$0.07  0.979$\pm$0.002   
20%          F-1 score   SVM         0.564$\pm$0.096  0.982$\pm$0.002   
                         LR          0.628$\pm$0.091  0.981$\pm$0.001   
                         DCT          0.301$\pm$0.18  0.978$\pm$0.001   
             sensitivity SVM         0.425$\pm$0.108  0.984$\pm$0.001   
                         LR          0.491$\pm$0.107  0.983$\pm$0.001   
                         DCT         0.213$\pm$0.121  0.977$\pm$0.001   
30%          F-1 score   SVM         0.553$\pm$0.068  0.981$\pm$0.003   
                         LR          0.629$\pm$0.086  0.982$\pm$0.002   
                         DCT          0.39$\pm$0.179  0.976$\pm$0.002   
             sensitivity SVM          0.41$\pm$0.064  0.983$\pm$0.002   
                         LR          0.491$\pm$0.105  0.983$\pm$0.001   
                         DCT          0.28$\pm$0.133  0.975$\pm$0.003   
40%          F-1 score   SVM         0.631$\pm$0.078  0.979$\pm$0.002   
                         LR          0.712$\pm$0.097  0.981$\pm$0.002   
                         DCT          0.213$\pm$0.16  0.973$\pm$0.003   
             sensitivity SVM         0.492$\pm$0.086  0.982$\pm$0.002   
                         LR          0.589$\pm$0.123  0.983$\pm$0.002   
                         DCT         0.151$\pm$0.108  0.972$\pm$0.004   

                                               SmtNN           SVMSmt  \
missing rate metric      classifier                                     
0%           F-1 score   SVM         0.984$\pm$0.002  0.985$\pm$0.002   
                         LR          0.983$\pm$0.003  0.982$\pm$0.002   
                         DCT         0.982$\pm$0.002  0.981$\pm$0.002   
             sensitivity SVM         0.985$\pm$0.002  0.986$\pm$0.002   
                         LR          0.985$\pm$0.002  0.984$\pm$0.002   
                         DCT         0.983$\pm$0.002  0.981$\pm$0.002   
10%          F-1 score   SVM         0.982$\pm$0.002  0.983$\pm$0.002   
                         LR          0.982$\pm$0.002  0.982$\pm$0.002   
                         DCT         0.981$\pm$0.002  0.979$\pm$0.001   
             sensitivity SVM         0.983$\pm$0.001  0.984$\pm$0.002   
                         LR          0.983$\pm$0.002  0.983$\pm$0.002   
                         DCT         0.982$\pm$0.002  0.979$\pm$0.001   
20%          F-1 score   SVM         0.982$\pm$0.002  0.983$\pm$0.001   
                         LR          0.982$\pm$0.002  0.982$\pm$0.001   
                         DCT          0.98$\pm$0.001  0.978$\pm$0.002   
             sensitivity SVM         0.984$\pm$0.001  0.984$\pm$0.001   
                         LR          0.983$\pm$0.001  0.983$\pm$0.001   
                         DCT         0.982$\pm$0.002  0.978$\pm$0.002   
30%          F-1 score   SVM         0.982$\pm$0.002  0.982$\pm$0.003   
                         LR          0.983$\pm$0.002  0.982$\pm$0.002   
                         DCT         0.981$\pm$0.003  0.977$\pm$0.003   
             sensitivity SVM

In [6]:
concat_to_export = pd.concat((res_all[0][0][0][0], res_all[0][0][0][1], 
                        res_all[1][0][0][0], res_all[1][0][0][1], 
                        res_all[2][0][0][0], res_all[2][0][0][1], 
                        res_all[3][0][0][0], res_all[3][0][0][1], 
                        res_all[4][0][0][0], res_all[4][0][0][1]))
concat_to_export.to_csv("mean_f1_sensi_mammography-low.csv", index = False)

In [7]:
print(all_res.to_latex(index = True, 
                       formatters = {"name": str.upper},
                      float_format = "{.1f}".format))

\begin{tabular}{lllllllll}
\toprule
    &             &     &         NearMiss &         kmeanSmt &            SmtNN &           SVMSmt &          InsHard &            OsImp \\
missing rate & metric & classifier &                  &                  &                  &                  &                  &                  \\
\midrule
0\% & F-1 score & SVM &  0.565\$\textbackslash pm\$0.045 &  0.983\$\textbackslash pm\$0.002 &  0.984\$\textbackslash pm\$0.002 &  0.985\$\textbackslash pm\$0.002 &   0.98\$\textbackslash pm\$0.003 &  0.984\$\textbackslash pm\$0.002 \\
    &             & LR &  0.607\$\textbackslash pm\$0.036 &  0.982\$\textbackslash pm\$0.003 &  0.983\$\textbackslash pm\$0.003 &  0.982\$\textbackslash pm\$0.002 &  0.977\$\textbackslash pm\$0.003 &  0.982\$\textbackslash pm\$0.002 \\
    &             & DCT &  0.447\$\textbackslash pm\$0.093 &  0.981\$\textbackslash pm\$0.002 &  0.982\$\textbackslash pm\$0.002 &  0.981\$\textbackslash pm\$0.002 &  0.787\$\textbackslash pm